# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!{sys.executable} -m spacy download en

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     ---------------------------------------- 0.1/12.8 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/12.8 MB 1.8 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.8 MB 3.6 MB/s eta 0:00:04
     ----- ---------------------------------- 1.9/12.8 MB 7.2 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 10.4 MB/s eta 0:00:01
     -------------- ------------------------- 4.6/12.8 MB 12.9 MB/s eta 0:00:01
     ----------------- ---------------------- 5.6/12.8 MB 14.4 MB/s eta 0:00:01
     ----------------- ---------------------- 5.6/12.8 M

"{sys.executable}" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
import re
import spacy
import catboost
from catboost import CatBoostClassifier, Pool
import torch
import transformers as ppb
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import GradientBoostingClassifier
from scipy import spatial #?
from tqdm import tqdm



In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Иван\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Загрузка данных

In [6]:
try:
    data = pd.read_csv('C:\\Users\\Иван\\Downloads\\toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [9]:
#удаляем не информативный столбец
data = data.drop('Unnamed: 0', axis = 1)
# приводим все символы к нижнему регистру
data['text'] = data['text'].str.lower()

In [10]:
#data = data.sample(n=1500, random_state=0)

## spaCy лемматтизация

Подготовим данные для линейной регресии и градиентного бустинга

In [11]:
# загружаем модель английского языка

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# грузит 20 минут
#nlp = spacy.load('en_core_web_sm')

In [12]:
# Определяем функцию для лемматизации текста
def lemma_text(text):

    # Производим разбор текста моделью spaCy
    doc = nlp(text)

    # Инициализируем список для лемм
    lemmas = []

    # Проходимся по токенам в тексте
    for token in doc:

        # Извлекаем лемму токена
        lemma = token.lemma_

        # Добавляем лемму в список
        lemmas.append(lemma)

    # Возвращаем пространством разделенную строку из лемм
    return " ".join(lemmas)

Протестируем корректность работы функций

In [13]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)

                                                text
0  The striped bats are hanging on their feet for...
1      you should be ashamed of yourself went worked


In [14]:
# Применяем функцию лемматизации к столбцу `text`
df_my['text'] = df_my['text'].apply(lemma_text)
print(df_my)

                                            text
0  the stripe bat be hang on their foot for good
1      you should be ashamed of yourself go work


In [15]:
%%time
tqdm.pandas()
data['text'] = data['text'].progress_apply(lemma_text)

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [13:27<00:00, 197.16it/s]

CPU times: total: 13min 21s
Wall time: 13min 27s


Таким образом мы получаем предобработанные текстовые данные, готовые для последующего использования в моделях ML, например для векторизации с помощью TF-IDF или Word2Vec

## Stopwords

Также избавимся от стоп слов

* Создаем список cleaned_text для хранения текста без стоп-слов
* Проходимся по каждому предложению
* Разбиваем на слова и фильтруем стоп-слова
* Объединяем обратно в строку
* Записываем результат в столбец 'text'

Теперь в столбце 'text' текст готов для векторизации - приведен к леммам и удалены стоп-слова.

In [16]:
stop_words = set(stopwords.words('english'))
cleaned_text = []
for text in data['text']:
    filtered_words = [word for word in text.split() if word not in stop_words]
    cleaned_text.append(" ".join(filtered_words))

data['text'] = cleaned_text

## Разбиение данных

In [17]:
train, final_test = train_test_split(
    data,
    train_size=0.9,
    random_state=0,
    stratify=data['toxic'])

y_train, X_train = \
    train['toxic'], train.drop(['toxic'], axis=1)
y_final_test, X_final_test = \
    final_test['toxic'], final_test.drop(['toxic'], axis=1)

## Дисбаланс классов

In [18]:
data['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

В наших данных наблюдается сильный дисбаланс классов, вычислим class_weight чтобы в дальнейшем использовать его при обучении моделей, придав больший вес ошибкам меньшего класса и меньший вес ошибкам большего класса.

In [19]:
# Преобразуем y_train в массив numpy
y_train_arr = np.array(y_train)

# Преобразуем список классов в array
classes_arr = np.array([0, 1])

# Вычисляем веса
weights = compute_class_weight('balanced', classes=classes_arr, y=y_train_arr)

# Создаем словарь
class_weight = dict(zip([0, 1], weights))

## Векторизация TF-IDF

In [20]:
%%time
# векторизация текста с помощью TF-IDF
vectorizer_TF = TfidfVectorizer()
X_train_TF = vectorizer_TF.fit_transform(X_train['text'])
#X_test_TF = vectorizer_TF.transform(X_test['text'])

CPU times: total: 4.16 s
Wall time: 4.16 s


По итогу первого раздела мы:

* Загрузили данные
* Привели текст к нижнему регистру
* Лемматизизровали слова
* Удалили стоп-слов
* Разбили полученные векторы на обучающую и тестовую выборки
* Взвесили классы
* Векторизировали текста с помощью TF-IDF


## Обучение

### LogisticRegression_TF

In [21]:
%%time

# Определяем словарь с параметрами, которые хотим перебрать

param_grid = {'C': [5,10],
              'max_iter': [400],
              'solver': ['saga'],
              'penalty': ['l2']}

# Создаем объект GridSearchCV с моделью LogisticRegression, словарем параметров, 3 фолдами для кросс-валидации и метрикой f1
grid_search = GridSearchCV(LogisticRegression(class_weight=class_weight), param_grid, cv=2, scoring='f1', n_jobs=-1)

# Обучаем объект GridSearchCV на обучающей выборке
grid_search.fit(X_train_TF, y_train)

# Выводим на экран лучшие параметры и лучшее значение f1
print('Лучшие параметры:', grid_search.best_params_)
print('Лучшее значение f1:', grid_search.best_score_)

Лучшие параметры: {'C': 10, 'max_iter': 400, 'penalty': 'l2', 'solver': 'saga'}
Лучшее значение f1: 0.7666523924156732
CPU times: total: 39.2 s
Wall time: 1min 10s


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### GradientBoostingClassifier_TF

Алгоритм gradient boosting справляется с дисбалансом таргета своими силами, поэтому веса классов передавать не нужно

In [22]:
%%time

# Определяем словарь с параметрами для GridSearchCV
param_grid = {'n_estimators': [10],
             'learning_rate': [0.2],
              'max_depth': [10],
              'subsample': [1.0]}

# Создаем модель GradientBoostingClassifier
model = GradientBoostingClassifier()

# Создаем объект GridSearchCV с полосой загрузки
grid_search = GridSearchCV(model, param_grid, cv=2, scoring='f1', n_jobs=-1, verbose=1)

# Обучаем модель на тренировочных данных
grid_search.fit(X_train_TF, y_train)

# Выводим лучшие параметры и значение метрики
print('Лучшие параметры:', grid_search.best_params_)
print('Лучшее значение F1:', grid_search.best_score_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
Лучшие параметры: {'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 10, 'subsample': 1.0}
Лучшее значение F1: 0.6198469748165162
CPU times: total: 2min
Wall time: 4min 26s


### CatBoostClassifier

Для CatBoost добавим валидационную выборку

In [23]:
# Деление тренировочного набора на тренировочный и валидационный наборы
X_train_cat, X_valid_cat, y_train_cat, y_valid_cat = train_test_split(
    X_train,
    y_train,
    train_size=0.8,
    random_state=0,
    stratify=y_train)

В CatBoost для обучения модели и оценки качества используются специальные объекты - Pool.

При создании Pool указываются:

* data - признаки объектов (X)
* label - целевые значения (y)
* text_features - список текстовых признаков

In [24]:
train_pool = Pool(
    data=X_train_cat,
    label=y_train_cat,
    text_features=['text']
)
valid_pool = Pool(
    data=X_valid_cat,
    label=y_valid_cat,
    text_features=['text']
)

In [25]:
#**kwargs - это синтаксис в Python для передачи произвольных именованных аргументов в функцию.
#В данном случае в kwargs передаются параметры для настройки обработки текста:
# tokenizers - настройки токенизатора для разбиения текста на токены
# dictionaries - создание словаря токенов
# feature_calcers - выбор метода для расчёта признаков (здесь Bag of Words)


def fit_model(train_pool, valid_pool, **kwargs):
    model = CatBoostClassifier(
        #CatBoost умеет запускать обучения на GPU
        #task_type='GPU',
        iterations=5000,
        eval_metric='F1',
        od_type='Iter',
        od_wait=500,
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=valid_pool,
        verbose=1000,
        #удобная фича CatBoost - возможность визуализировать процесс обучения в виде графиков
        plot=True,
        use_best_model=True)

Модель CatBoost умеет токенизировать и векторизировать сырой текст, поэтому текста в числовые признаки можно выполнить непосредственно с помощью методов CatBoost, без использования отдельных алгоритмов вроде мешка слов или tf-idf.

In [26]:
model_cat = fit_model(
    train_pool, valid_pool,
    learning_rate=0.35,
    tokenizers=[
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }
    ],
    dictionaries = [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    feature_calcers = [
        'BoW:top_tokens_count=10000'
    ]
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.4328358	test: 0.4332454	best: 0.4332454 (0)	total: 270ms	remaining: 22m 27s
1000:	learn: 0.8741672	test: 0.7781387	best: 0.7795185 (965)	total: 2m 25s	remaining: 9m 42s
2000:	learn: 0.9452883	test: 0.7826581	best: 0.7841060 (1918)	total: 4m 52s	remaining: 7m 18s
3000:	learn: 0.9657516	test: 0.7872060	best: 0.7887801 (2926)	total: 7m 22s	remaining: 4m 54s
4000:	learn: 0.9747700	test: 0.7914719	best: 0.7931163 (3642)	total: 9m 55s	remaining: 2m 28s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.7931163487
bestIteration = 3642

Shrink model to first 3643 iterations.


## BERT

* Для BERT будем использовать сэмпл, вместо всего датафрейма
* BERT так же не требует лемматизации и токинезации

Мы будем использовать 2 модели

* DistilBERT – это сжатая версия популярное архитектуры BERT

In [27]:
X_train = X_train.sample(n=1000, random_state=0,replace=False)

In [28]:
y_train = y_train.loc[X_train.index]

### Загрузка предобученного DistilBERT

In [29]:
# Создаём объекты классов DistilBERT + DistilBERTTokenizer
model_class, tokenizer_class = ppb.DistilBertModel, ppb.DistilBertTokenizer

In [30]:
# Загружаем веса
tokenizer = tokenizer_class.from_pretrained('distilbert-base-uncased')
model = model_class.from_pretrained('distilbert-base-uncased')

In [31]:
pattern = re.compile('<.*?>')
X_train['text'] = X_train['text'].apply(lambda line: pattern.sub('', line))


In [32]:
tokenized = X_train['text'].apply((lambda x: tokenizer.encode(x,
                                                           add_special_tokens=True,
                                                           truncation=True,
                                                           max_length=128)))

In [33]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

In [34]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 128)

In [35]:
%%time

# тензор с токенизированным текстом, преобразованным в числовые идентификаторы токенов
input_ids = torch.tensor(padded)
# маска внимания
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():# отключает трекинг истории вычислений
    last_hidden_states = model(                #метод модели BERT, который принимает на вход тензоры input_ids и attention_mask.
        input_ids, attention_mask=attention_mask)

CPU times: total: 10min 7s
Wall time: 1min 16s


### Получение признаков (эмбеддингов)

In [36]:
features = last_hidden_states[0][:,0,:].numpy()


In [37]:
X_train_bert, X_valid_bert, y_train_bert, y_valid_bert = train_test_split(features,y_train, test_size=0.25)


In [38]:
lr_clf = LogisticRegression(max_iter= 500)
lr_clf.fit(X_train_bert, y_train_bert)

LogisticRegression(max_iter=500)

In [39]:
pred_bert_lr = lr_clf.predict(X_valid_bert)
f1 = f1_score(y_valid_bert, pred_bert_lr)
print(f1)

0.6486486486486486


## Тест

Лучшую метрику показала модель CatBoosting, запустим ее на тестовых данных.

In [40]:
test_pool = Pool(
    data=X_final_test,
    label=y_final_test,
    text_features=['text']
)

In [41]:
preds = model_cat.predict(test_pool)

In [42]:
f1 = f1_score(y_final_test, preds, average='macro')
print(f1)

0.8796397027493748


## Итог


В рамках проекта для интернет-магазина "Викишоп" была поставлена задача классификации токсичных комментариев пользователей.

Для решения задачи были выполнены следующие шаги:

* Предобработка текста: лемматизация с помощью spaCy, удаление стоп-слов
* Разбиение данных на обучающую и тестовую выборки
* Борьба с дисбалансом классовoversampling
* Векторизация текста: TF-IDF
* Обучение моделей: логистическая регрессия, градиентный бустинг, CatBoost , BERT

Оценка метрик качества на тестовой выборке больше 0.75 показала модель CatBoost

По результатам исследования остроена модель классификации токсичных комментариев на основе CatBoost, позволяющая с фильтровать нежелательный контент с достаточным качеством F1 больше 0.75. Результат удовлетворяет условиям проекта.